In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
from scipy.integrate import simpson
from scipy.stats import median_abs_deviation

In [ ]:
## Load everything
pm_data = np.load('npy/pm_eLAE_15deg_uniform.npy')
pm_data_noerr = np.load('npy/pm_LAE_15deg_uniform.npy')
f_name = 'Source_cats/Source_cat_LAE_15deg_uniform.npy'
mock = np.load(f_name, allow_pickle=True).item()

errors = np.load('npy/median_errors.npy')

pm_err = (np.ones(pm_data.shape).T * errors).T

filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
# EW diff
w_lya = 1215.67

for N_nb in range(1, 11):
    fig, ax = plt.subplots(figsize=(12, 9))
    EW_ret_Arr = np.ones(mock['EW_Arr'].shape) * -999.
    for nb_c in [13]:
        fwhm = nb_fwhm(tcurves, nb_c, True)
        cont_est, cont_err = stack_estimation(
            pm_data, pm_err, nb_c, N_nb, w_central
        )

        EW_ret = (pm_data[nb_c] - cont_est) * fwhm / cont_est\
            / (w_central[nb_c] / w_lya)
        line = (
            (pm_data[nb_c] - cont_est > 3 * (pm_err[nb_c]**2 + cont_err**2)**0.5)
            & (EW_ret > 10)
        )
        EW_ret_Arr[line] = EW_ret[line]

    mask999 = np.where(EW_ret_Arr != -999.)

    ax.plot(
        mock['EW_Arr'][mask999], EW_ret_Arr[mask999],
        marker='.', markersize=12, linestyle='', alpha=0.7
        )
    ax.plot(
        np.linspace(0, 300, 300), np.linspace(0, 300, 300), linestyle='--'
        )
    ax.text(
        25, 250, 'bias = {0:0.2f} \nsigma = {1:0.2f}'.format(
            np.median(
                EW_ret_Arr[mask999] - mock['EW_Arr'][mask999],
                ),
            median_abs_deviation(
                EW_ret_Arr[mask999] - mock['EW_Arr'][mask999],
                ),
            ),
        fontsize=15
    )

    ax.set_ylim((0, 300))
    ax.set_xlim((0, 300))
    ax.set_xlabel('Real EW$_0$ ($\AA$)', fontsize=15)
    ax.set_ylabel('Retrieved EW$_0$ ($\AA$)', fontsize=15)
    ax.set_title('N_nb = {}'.format(N_nb), fontsize=20)
    plt.show()